<h4>Paso 1: Importo Librerias</h4>

In [10]:
import geopy as gp
from geopy.geocoders import Nominatim
from geopy.exc import GeopyError
import folium as f
import pandas as pd
import numpy as np
import time as t

<h4>Paso 2: Cargo CSV</h4>

In [11]:
df_top25 = pd.read_csv("top_25.csv", header = None)
df_top25.columns = ["street", "repeticiones"]

<h4>Paso 3: Divido en campo Calle y Altura</h4>

In [12]:
df_domicilios = pd.DataFrame(columns = ["Calle", "Altura", "Repeticiones", "Latitud", "Longitud"])
df_domicilios["Calle"] = df_domicilios["Calle"].astype(str)
df_domicilios["Altura"] = df_domicilios["Altura"].astype(np.int32)
df_domicilios["Repeticiones"] = df_domicilios["Repeticiones"].astype(np.int32)
df_domicilios["Latitud"] = df_domicilios["Latitud"].astype(np.float32)
df_domicilios["Longitud"] = df_domicilios["Longitud"].astype(np.float32)

for index, row in df_top25.iterrows():
    domicilio = row["street"]
    repeticiones = int(row["repeticiones"])
    pos = domicilio.find("NRO ")
    calle = domicilio[0:pos]
    altura = int(domicilio[pos + 4:])
    df_domicilios = df_domicilios.append({"Calle" : calle, "Altura" : altura, "Repeticiones" : repeticiones}, ignore_index = True)


<h4>Paso 4: Calculo latitud y longitud para cada Domicilio</h4>

In [13]:
geolocator = Nominatim(user_agent = "ej2_dom")

for index, row in df_domicilios.iterrows():
    try:
        direccion = str(row["Altura"]) + " " + str(row["Calle"]).strip().lower() + ", ciudad autonoma de buenos aires, argentina"
        
        location = geolocator.geocode(direccion, addressdetails = True)

        location_dict = location.raw

        for key, value in location_dict.items():
            if key == "lat":
                latitud = str(value)

            if key == "lon":
                longitud = str(value)
        
        df_domicilios.loc[index, "Latitud"] = float(latitud)
        df_domicilios.loc[index, "Longitud"] = float(longitud)
        
        # La API gratuita no me permite más de 1 request por segundo
        t.sleep(1)

    except (GeopyError, AttributeError):
        print("Domicilio " + direccion + " no geolocalizado")

<h4>Paso 5: Geolocalizo los Domicilios en un Mapa</h4>

In [14]:
mapa_caba = f.Map(location = [-34.60, -58.45], zoom_start = 12)

dom_mark = f.map.FeatureGroup()

for index, row in df_domicilios.iterrows():
    dom_mark.add_child(
        f.CircleMarker(
            [row["Latitud"], row["Longitud"]],
            radius = int(row["Repeticiones"] / 300000),
            color = 'yellow',
            fill = True,
            fill_color = 'blue',
            fill_opacity = 0.6
        )
    )
    
    f.Marker([row["Latitud"], row["Longitud"]], popup = row["Calle"] + str(row["Altura"])).add_to(mapa_caba) 

mapa_caba.add_child(dom_mark)

mapa_caba